# RL 训练

In [2]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import scipy.io
from matplotlib.gridspec import GridSpec
import math

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.logger import configure
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO 

import sys
import os

# 将 src 目录添加到模块搜索路径
sys.path.append(os.path.abspath('src'))

# 导入 IEMEnv 类
from src.envs.iseec_lx_v4_mdp_plot import IEMEnv

定义保存位置参数

In [ ]:
# 设置日志保存路径和格式
tmp_path = "logs/sb3_log/"
new_logger = configure(
    tmp_path, 
    ["stdout",     # 终端输出
     "csv",        # CSV文件
     "tensorboard", # Tensorboard格式
     "json"        # JSON格式
    ]
)

### 定义环境

In [5]:
env = IEMEnv(reward_type="PB_ste")
env_monitor = Monitor(env, './logs/monitor_logs/monitor2')



[1850 1851 1852 1853 1854 1855 1856 1857 1858 1859 1860 1861 1862 1863
 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877
 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891
 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905
 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919
 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933
 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947
 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961
 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975
 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031
 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045
 2046 

### 定义算法

In [ ]:
# 定义模型
model = PPO(
    "MlpPolicy", 
    env_monitor, 
    verbose=1,
    tensorboard_log="./logs/tensorboard_logs",
    )
model.set_logger(new_logger) # set the parameter of the logger

### 模型训练

In [ ]:
model.learn(
    total_timesteps=int(2e4),
    tb_log_name="iseec_v4_PPO_Net256_2e4",
            )

In [ ]:
# 模型保存
model.save("iseec_v4_PPO_Net256_2e4")

### 评估模型训练结果

> 利用 evaulate policy 函数评估

# 加载训练好的 RL 模型

In [3]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(
    model, 
    model.get_env(), 
    deterministic=True,
    render=True,
    n_eval_episodes=10)

NameError: name 'model' is not defined

In [105]:
print(mean_reward)
print(std_reward)

0.0
0.0


### 结果展示

In [6]:
episodes = 30
max_steps = 251

for ep in range(episodes):
    obs, _ = env.reset()
    done = False
    
    for i in range(max_steps):
        action = env.action_space.sample()
        # action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, info = env.step(action)
        
        # 每10步更新一次图像
        if ep % 10 == 0:
            env.render()
            # plt.pause(0.01)
        
        reward += reward
    print(f"Episode {ep} finished with reward{reward}")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


KeyboardInterrupt: 

# 保存运行结果